<a href="https://colab.research.google.com/github/aaryan19/sentiment_anaysis/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. p.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Bidirectional
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/pr.csv")
# Keeping only the neccessary columns
data = data[['review','score']]

In [ ]:
max_fatures = 2000
maxlen = 200
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['review'].values)
X = tokenizer.texts_to_sequences(data['review'].values)
X = pad_sequences(X)

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
# Input for variable-length sequences of integers
#inputs = keras.Input(shape=(None,), dtype="int32")
# Embed each integer in a 128-dimensional vector
#x = layers.Embedding(max_features, 128)(inputs)
# Add 2 bidirectional LSTMs
#x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
#x = layers.Bidirectional(layers.LSTM(64))(x)
# Add a classifier
#outputs = layers.Dense(1, activation="sigmoid")(x)
#model = keras.Model(inputs, outputs)
#model.summary()

In [ ]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
#model.add(Bidirectional(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2,return_sequences = True)))
#model.add(Bidirectional(LSTM(10)))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 651, 128)          256000    
                                                                 
 spatial_dropout1d (SpatialD  (None, 651, 128)         0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 196)               254800    
                                                                 
 dense (Dense)               (None, 2)                 394       
                                                                 
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
a =[]
for i in data["score"]:
  if i > 3:
    a.append("positive")
  else:
    a.append("negative")
data["score"] = a
data.head()

,review,score
0,"['bought', 'sever', 'vital', 'can', 'dog', 'fo...",positive
1,"['product', 'arriv', 'label', 'jumbo', 'salt',...",negative
2,"['confect', 'around', 'centuri', 'light', 'pil...",positive
3,"['look', 'secret', 'ingredi', 'robitussin', 'b...",negative
4,"['great', 'taffi', 'great', 'price', 'wide', '...",positive


In [ ]:
Y = pd.get_dummies(data['score']).values
Y
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.3, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

array([[0, 1],
       [1, 0],
       [0, 1],
       ...,
       [0, 1],
       [0, 1],
       [0, 1]], dtype=uint8)

In [ ]:
batch_size = 32
#model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
model.fit(X_train, Y_train, epochs = 8, batch_size=batch_size, verbose = 2)

Epoch 1/8
1094/1094 - 3389s - loss: 0.3658 - accuracy: 0.8425 - 3389s/epoch - 3s/step
Epoch 2/8
1094/1094 - 3366s - loss: 0.2979 - accuracy: 0.8759 - 3366s/epoch - 3s/step
Epoch 3/8
1094/1094 - 3392s - loss: 0.2738 - accuracy: 0.8877 - 3392s/epoch - 3s/step
Epoch 4/8
1094/1094 - 3386s - loss: 0.2556 - accuracy: 0.8962 - 3386s/epoch - 3s/step
Epoch 5/8
1094/1094 - 3381s - loss: 0.2379 - accuracy: 0.9033 - 3381s/epoch - 3s/step
Epoch 6/8
1094/1094 - 3382s - loss: 0.2233 - accuracy: 0.9107 - 3382s/epoch - 3s/step
Epoch 7/8
1094/1094 - 3380s - loss: 0.2087 - accuracy: 0.9153 - 3380s/epoch - 3s/step
Epoch 8/8
1094/1094 - 3380s - loss: 0.1936 - accuracy: 0.9249 - 3380s/epoch - 3s/step


In [ ]:
validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

422/422 - 168s - loss: 0.3725 - accuracy: 0.8658 - 168s/epoch - 397ms/step
score: 0.37
acc: 0.87


In [ ]:
pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0
for x in range(len(X_validate)):
    
    result = model.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 2)[0]
   
    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0:
            neg_correct += 1
        else:
            pos_correct += 1
       
    if np.argmax(Y_validate[x]) == 0:
        neg_cnt += 1
    else:
        pos_cnt += 1



print("pos_acc", pos_correct/pos_cnt*100, "%")
print("neg_acc", neg_correct/neg_cnt*100, "%")

1/1 - 0s - 458ms/epoch - 458ms/step
1/1 - 0s - 172ms/epoch - 172ms/step
1/1 - 0s - 179ms/epoch - 179ms/step
1/1 - 0s - 183ms/epoch - 183ms/step
1/1 - 0s - 193ms/epoch - 193ms/step
1/1 - 0s - 199ms/epoch - 199ms/step
1/1 - 0s - 170ms/epoch - 170ms/step
1/1 - 0s - 170ms/epoch - 170ms/step
1/1 - 0s - 171ms/epoch - 171ms/step
1/1 - 0s - 187ms/epoch - 187ms/step
1/1 - 0s - 168ms/epoch - 168ms/step
1/1 - 0s - 169ms/epoch - 169ms/step
1/1 - 0s - 166ms/epoch - 166ms/step
1/1 - 0s - 170ms/epoch - 170ms/step
1/1 - 0s - 185ms/epoch - 185ms/step
1/1 - 0s - 164ms/epoch - 164ms/step
1/1 - 0s - 179ms/epoch - 179ms/step
1/1 - 0s - 172ms/epoch - 172ms/step
1/1 - 0s - 173ms/epoch - 173ms/step
1/1 - 0s - 178ms/epoch - 178ms/step
1/1 - 0s - 170ms/epoch - 170ms/step
1/1 - 0s - 169ms/epoch - 169ms/step
1/1 - 0s - 166ms/epoch - 166ms/step
1/1 - 0s - 162ms/epoch - 162ms/step
1/1 - 0s - 181ms/epoch - 181ms/step
1/1 - 0s - 174ms/epoch - 174ms/step
1/1 - 0s - 175ms/epoch - 175ms/step
1/1 - 0s - 167ms/epoch - 167

In [ ]:
import joblib
filename = 'final_model1.sav'
joblib.dump(model, filename)

INFO:tensorflow:Assets written to: ram://849e69a3-47fb-45cd-bcbe-cc8e8584aea4/assets


['final_model1.sav']

In [ ]:
filename = 'final_model1.h5'
joblib.dump(model, filename)

INFO:tensorflow:Assets written to: ram://4926cb2a-4847-4fc5-8f5c-966ea00481ee/assets


INFO:tensorflow:Assets written to: ram://4926cb2a-4847-4fc5-8f5c-966ea00481ee/assets


['final_model1.h5']